In [1]:
from ngsolve import *

In [2]:
import numpy as np
import netgen.meshing as ngm
from netgen.csg import *
from netgen.meshing import MeshingStep
from ngsolve.comp import IntegrationRuleSpaceSurface

In [3]:
from es_utils import pos_transformer

In [4]:
from geometry import *

In [5]:
from esfem import WillMoreMDR

In [6]:
from ngsolve.webgui import Draw

In [7]:
from netgen.occ import SplineApproximation, Pnt, Axis, Face, Wire, Segment, Revolve, OCCGeometry, Z, X, Y
#%% Construction of Initial Curved Mesh
dim = 3
order = 2
msize = 0.1
dt = Parameter(0)

In [8]:
from ngsolve import Mesh

In [9]:
def GetRotFace(CurveFunc,T_min=-np.pi/2,T_max=np.pi/2,n=100):
    '''
        Generate Rotational Mesh by revolving a 2d curve (CurveFunc) around an axis (axis)
        CurveFunc: function of phi, return (x(phi), 0, z(phi)) for example
        n: number of points on the curve
        T_min, T_max: parameter range of the curve
        msize: mesh size
        return: mesh 
    '''
    pnts = [CurveFunc(phi) for phi in np.linspace(T_min,T_max,n)]
    spline = SplineApproximation(pnts, tol=1e-4)
    f = Face(Wire([spline]))
    return f,spline

* 先生成母线的参数方程 on（x-z平面），然后旋转

In [10]:
def Egg(phi,r = 0.5): 
    z = 0.3-r*np.cos(phi)
    res = Pnt(r*np.sin(phi), 0, z)
    return res

In [11]:
f,spline = GetRotFace(Egg,T_min=np.arctan2(4,3),T_max=np.pi)

In [12]:
spline.edges[0].vertices[0].maxh = 0.05

In [13]:
seg3 = Segment(Pnt(0, 0, 0), spline.edges[0].vertices[0].vertices[0].p)
seg4 = Segment(Pnt(0, 0, 0), spline.edges[0].vertices[1].vertices[0].p)
f = Face(Wire([seg3,spline, seg4]))

In [14]:
spline.edges[0].vertices[1].vertices[0]

In [15]:
Draw(f)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

BaseWebGuiScene

In [16]:
torus = f.Revolve(Axis((0,0,0), Z), 180)
mesh = Mesh(OCCGeometry(torus).GenerateMesh(maxh=msize,grading=0.5))

In [17]:
Coords = np.array([v.point for v in mesh.vertices])

In [18]:
mesh.ne

891

In [19]:
Draw(x,mesh,"vv")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [274]:
mesh = mesh.ngmesh.Mirror((0,0,0),(0,-1,0))

In [275]:
mymesh = Mesh(mesh)

In [276]:
Draw(mymesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [277]:
Coords2 = np.array([v.point for v in mymesh.vertices])

In [296]:
mymesh.ne

1782

### 根据比y大的点的个数可以看出来中间的点没有影响

In [310]:
np.sum((Coords[:,1])>-1e-3)

286

In [309]:
np.sum((Coords2[:,1])>-1e-3)

286